# Introduction

The aim is to build a part-based Active Appearance Model (AAM) in order to model the human pose. AAMs are generative statistical models of the shape and appearance of an object. Given the articulated nature of human pose, the appearance must be represented in a part-based fashion. Thus, in here, we implement the following paper:

* G. Tzimiropoulos, M. Pantic. _"Gauss-Newton Deformable Part Models for Face Alignment in-the-Wild"_, CVPR 2014. [[pdf]](http://ibug.doc.ic.ac.uk/media/uploads/documents/tzimiro_pantic_cvpr_2014.pdf)

Other relative bibliography on AAMs is:

* E. Antonakos, J. Alabort-i-medina, G. Tzimiropoulos, S. Zafeiriou. _"Feature-Based Lucas-Kanade and Active Appearance Models"_, TIP 2015. [[pdf]](http://ibug.doc.ic.ac.uk/media/uploads/documents/antonakos2015feature.pdf)
* I. Matthews, S. Baker. _"Active appearance models revisited"_, IJCV 2004. [[pdf]](http://www-cgi.cs.cmu.edu/afs/cs.cmu.edu/Web/People/efros/courses/AP06/Papers/matthews_ijcv_2004.pdf)
* T.F. Cootes, G.J. Edwards, C.J. Taylor. _"Active appearance models"_, T-PAMI 2001. [[pdf]](http://www.cs.cmu.edu/~efros/courses/AP06/Papers/cootes-pami-01.pdf)

The rest of the notebook is structured as follows:

1. Import neccessary functionality
2. Load training and testing data
3. Train AAM
4. Fit AAM
5. Visualize results
6. Summary and goals

__NOTE__: Before starting, make should that _menpo_, _menpofit_ and _alabortijcv2015_ are updated to the latest master.

### 1. Import neccessary functionality
Let's first import all the functions that are needed.

In [ ]:
# The purpose of this is for the visualized figures to be inline the browser
%matplotlib inline

# Import stuff from menpo
import menpo.io as mio
from menpo.feature import no_op, dsift, fast_dsift, double_igo
from menpo.visualize import visualize_images, visualize_pointclouds, print_dynamic, print_progress
from menpo.landmark import labeller, human36M_pose_32, human36M_pose_17

# Import stuff from menpofit
from menpofit.visualize import visualize_shape_model, visualize_fitting_result, plot_ced

# Import stuff from alabortijcv2015 (this will be soon integrated in menpofit)
from alabortijcv2015.aam import PartsAAMBuilder, PartsAAMFitter
from alabortijcv2015.aam.algorithm import SIC, BSC
from alabortijcv2015.utils import pickle_load, pickle_dump
from alabortijcv2015.result import SerializableResult

# Other stuff
import numpy as np
import re
from itertools import islice

Define general paths:

In [ ]:
path_to_training_images = '/vol/atlas/databases/body/Human3.6M/Trainig/BySubject/S1/data/Directions_1.54138969/'
path_to_testing_images = '/vol/atlas/databases/body/Human3.6M/Trainig/BySubject/S5/data/Directions_1.54138969/'
save_path = '/vol/atlas/homes/nontas/'

Define the landmarks mark-up that will be used. _Manpo_ has two labels related to the Human3.6M database:
* _human36M_pose_32_: This includes all the initial landmarks provided with the database. However, some points are defined more than once (duplicate) and some others are not very accurately annotated.
* _human36M_pose_17_: This is a subset of the 17 points that includes only uniquely and accurately defined points. I think it makes much more sense to use this mark-up in order to train a deformable model.

In [ ]:
group = human36M_pose_17 #human36M_pose_32
group_str = group.func_name

### 2. Load training and testing data
Now let's load our training data. The more training data that you use, the better model you will get. From my experience on face, I believe that about 1000-2000 training images would be enough. But I'm not sure about pose. You need to **experiment** with respect to:
* the number of training images
* the way to subset the training images to create models for different poses.

Let's first define a function for loading the data. Because the Human3.6 database in '/vol/atlas/databases/' has a specific structure, we need to define a function that will skip the '*_mask.png' images.

In [ ]:
def load_human36_images(path_to_training_images, crop_proportion, pattern, group, max_images=None):
    images = []

    paths_to_load = []
    for path in mio.image_paths(path_to_training_images + '*.png'):
        if pattern.match(path.name):
            paths_to_load.append(path_to_training_images + path.name)

    if max_images is None:
        max_images = len(paths_to_load)
        
    for path in print_progress(paths_to_load[:max_images]):
        im = mio.import_image(path)
        im.crop_to_landmarks_proportion_inplace(crop_proportion)
        if im.n_channels == 3:
            im = im.as_greyscale(mode='luminosity')
        labeller(im, 'PTS', group)
        images.append(im)
    
    return images

So we need to define the options and then load the training images

In [ ]:
crop_proportion = 0.4
# I only need the images with pattern "1234.png" and not "1234_mask.png"
pattern = re.compile('[0-9]{4}\.png')

training_images = load_human36_images(path_to_training_images, crop_proportion, pattern, group, max_images=None)

We can visualize the images using the widget. By pressing the Play button we can see the actual animation.

In [ ]:
visualize_images(training_images)

We can also visualize just the pointclouds using the respective widget

In [ ]:
training_pointclouds = [im.landmarks[group_str].lms for im in training_images]
visualize_pointclouds(training_pointclouds)

We can do the same for the testing images. _Note that the testing images must not be the same as the training ones_.

In [ ]:
crop_proportion = 0.4
pattern = re.compile('[0-9]{4}\.png')

testing_images = load_human36_images(path_to_testing_images, crop_proportion, pattern, group, max_images=5)

In [ ]:
visualize_images(testing_images)

### 3. Train AAM
In this step, we need to train the AAM. In general an AAM is trained in a multilevel fashion, i.e. in different resolutions. The parameters of the AAM builder are the following:

* **patch_shape**: The appearance model of the AAM will be obtained by sampling appearance patches with the specified shape around each landmark.
* **features**: The features to be used. _double_igo()_ return 4 channels, _dsift()_ 36 and _fast_dsift()_ 8.
* **diagonal**: During building an AAM, all images are rescaled to ensure that the scale of their landmarks matches the scale of the mean shape. The scale is defined through the diagonal. Note that because the reference frame is computed from the mean landmarks, this argument also specifies the diagonal length of the reference frame (provided that features computation does not change the image size).
* **normalize_parts**: Here we can pass in a function that will be apllied on the patches before extracting features (thus on their intensities values). The purpose of such a function would be to normalize the patch's values, e.g. by dividing with the values' norm.
* **scales**: This argument defines the number of levels as well as their resolutions. len(scales) is the number of levels. Then each value defines the scale of each level with respect to the original resolution.
* **max_shape_components**:  Defines the number of shape components to keep in memory. It is very important for memory efficiency.
* **max_appearance_components**: Defines the number of appearance components to keep in memory. It is very important for memory efficiency.

We select the following options and then train the model:

In [ ]:
patch_shape = (24, 24)
features = fast_dsift
diagonal = 150
normalize_parts = no_op
scales = (1, .5)
max_shape_components = 25
max_appearance_components = 150

In [ ]:
aam = PartsAAMBuilder(parts_shape=patch_shape,
                      features=features,
                      diagonal=diagonal,
                      normalize_parts=normalize_parts,
                      scales=scales,
                      max_shape_components=max_shape_components,
                      max_appearance_components=max_appearance_components).build(training_images, group=group_str, 
                                                                                 verbose=True)

The model can be saved as follows:

In [ ]:
aam_type = aam.__class__.__name__
pickle_dump(aam, save_path + aam_type + '_' + features.__name__ + '.pickle')

and then reloaded as:

In [ ]:
aam = pickle_load(save_path + 'PartsAAM_fast_dsift.pickle')

We can visualize the multilevel shape model using the widgets:

In [ ]:
visualize_shape_model(aam.shape_models)

We can also plot the eigenspectrums of the PCA models. This helps in selecting a proper number of components.

In [ ]:
aam.appearance_models[1].plot_eigenvalues_ratio_widget()

The above options play an important role to the performance of the model. We should experiment especially with **scales** and **patch_shape**.

### 4. Fit AAM
In order to fit the AAM, we first need to call the fitter. The options we need to define are:
* **algorithm_cls**: The optimization technique to be used. After my experiments, the Simultaneous (SIC) and Bayesian (BSC) are the most suitable.
* **n_shape**: The number of active shape components during fitting. It can be a list that specifies the components per level (low -> high).
* **n_appearance**: The number of active appearance components during fitting. It can be a list that specifies the components per level (low -> high).
* **sampling_mask (sampling_step)**: It defines the sampling points based on which the Jacobians will be computed. If _sampling_step == 1_, then there is no downsampling. The aim of this argument is mostly efficiency.
* **noise_std**: The fitting procedure requires an initial estimation of the landmarks' locations. This is usually acquired by applying an object detector and then aligning the mean shape in the returned bounding box. However, since we don't have yet a human body detector, we will initialize the optimization by adding some noise to the groundtruth landmarks (annotations). This argument defines the standard deviation of this noise. If _None_, then no noise will be applied.
* **seed**: This argument fixes the random generator engine to be used for the initialization noise. This is important since we fix the initial shapes, so we can run consistent expreriments.
* **rotation**: Specifies whether groundtruth in-plane rotation is to be used to produce the initial shape.
* **max_iters**: The maximum number of fitting iterations for all levels.
* **prior**: Flag that enables/disables a regularization on the Hessian matrix (adds values on its diagonal).

Let's define the options and create the fitter object:

In [ ]:
algorithm_cls = SIC  #BSC
n_shape = [5, 15]; 
n_appearance = [30, 50]
sampling_step = 1

noise_std = None #0.01
seed = 2
rotation = True
max_iters = 50
prior = False

In [ ]:
sampling_mask = np.require(np.zeros(patch_shape), dtype=np.bool)
sampling_mask[::sampling_step, ::sampling_step] = True

fitter = PartsAAMFitter(aam, algorithm_cls=algorithm_cls, n_shape=n_shape,
                        n_appearance=n_appearance, sampling_mask=sampling_mask)

Let's now fit the model to the test images. Each fitting returns a

In [ ]:
fitter_results = []

# Fix the random seed
np.random.seed(seed=seed)

# Fit model on each test image
for j, im in enumerate(testing_images):
    # Get groundtruth shape
    groundtruth_shape = im.landmarks[group_str].lms
    # Add noise if required
    if noise_std is not None:
        initial_shape = fitter.perturb_shape(groundtruth_shape, noise_std=noise_std, rotation=rotation)
    else:
        initial_shape = groundtruth_shape
    # Fit
    fr = fitter.fit(im, initial_shape, gt_shape=groundtruth_shape, max_iters=max_iters, prior=prior)
    # Append fitting result
    fr.downscale = 0.5
    fitter_results.append(fr)
    # Print progress
    print_dynamic("Image: {}/{}, Error: {:1.4f} -> {:1.4f}".format(j, len(testing_images)-1, 
                                                                   fr.initial_error(), fr.final_error()))

The results can be saved as:

In [ ]:
results = [SerializableResult('none', fr.shapes(), fr.n_iters, 'FastSIC', fr.gt_shape) 
           for fr in fitter_results]
pickle_dump(results, save_path + aam_type + '_' + features.__name__ + '_noise' + str(noise_std) + '.pickle')

The most important parameters of the fitting process are **n_shape** and **n_appearance**.

### 5. Visualize results
The results can be very easily visualized using the _visualize_fitting_result()_ widget. The widget can show the initial and final shapes, the iterations as well as plot the error graphs (CED).

In [ ]:
visualize_fitting_result(fitter_results)

The Cumulative Error Curve (CED) can also be separately plotted as:

In [ ]:
error_type = 'me_norm' #'me_norm', or 'me' or 'rmse'

initial_errors = [fr.initial_error(error_type=error_type) for fr in fitter_results]
final_errors = [fr.final_error(error_type=error_type) for fr in fitter_results]

plot_ced([initial_errors, final_errors])

You can also print the mean, median and standard deviation of the results:

In [ ]:
print("               |  mean  | median |  std  ")
print("Initialization | {:1.4f} | {:1.4f} | {:1.4f}".format(np.mean(initial_errors), 
                                                            np.median(initial_errors),
                                                            np.std(initial_errors)))
print("Fitting result | {:1.4f} | {:1.4f} | {:1.4f}".format(np.mean(final_errors), 
                                                            np.median(final_errors),
                                                            np.std(final_errors)))

### 6. Summary and goals
The purpose of this notebook is to be used as a general guide. So the selected parameter values are not optimized at all.

The most important parameters to optimize for, as mentioned before, are:
* _patch_shape_, _scales_, _n_shape_, _n_appearance_

On the other hand, parameters that also need some fixing are:
* _diagonal_, _features_, _max_iters_, _noise_std_

The most important things are:
1. _Always visualize your model to see if it makes sense_. Specifically use the _visualize_shape_model()_ widget to see the instances that it generates. Also plot its eigenspectrum to have better understanding about the number of components that should be kept.
2. _Always visualize your results_. The curves say nothing. You need to specifically visualize your initial shapes to see if they are too easy/hard. Of course, in a realistic scenario, the initial shapes should not be the groundtruth annotations! And then visualize the fitted shapes to see which error values actually correspond to good results. All these can be easily done using the _visualize_fitting_result()_ widget.